In [12]:
import numpy as np
import matplotlib.pyplot as plt

from threading import Thread

from io import StringIO
from struct import unpack
from copy import copy, deepcopy

import time

import sys
import threading
import visa
from socket import SOL_SOCKET, SO_KEEPALIVE
from ftplib import FTP, error_temp

# UI
# from traits.api import HasTraits, Range, Float, Bool, Int, Str, Enum, Button, Property, Instance, on_trait_change
# from traitsui.api import View, VGroup, HGroup, Item, TextEditor, EnumEditor
#from enthought.traits.api import HasTraits, Instance, Property, Range, Float, Int, String, Array, Enum, Tuple, Button, on_trait_change, cached_property, Bool, Str
                                 
#from enthought.traits.ui.api import View, Item, HGroup, Tabbed, VGroup, EnumEditor
#from enthought.enable.api import ComponentEditor, Component
#from enthought.chaco.api import Plot, ArrayPlotData
#from enthought.chaco.tools.api import PanTool, ZoomTool

In [10]:
from enthought.traits.api import HasTraits

ModuleNotFoundError: No module named 'enthought'

In [13]:
class Waveform(Thread):
    """ Compile a waveform from a sequence of Pulse objects upon instantiation.
        The wave form can be created as AWG-ready waveform file (*.WFM; 5 byte
        data width) or as pattern file (*.PAT; 2 byte width). Either file type
        can be handeled similar to an open file, i.e. be transmitted using 
        ftplib.FTP.storbinary.
        
        Constructor arguments:
        Waveform(name, pulse_seq[, offset[, sampling[, file_type]]])
        
        name        - string - file name, extension will be added automatically
        pulse_seq   - iterable of Pulse objects - sequence of pulses from which
                                                waveform will be compiled
        [offset]    - int - time offset [samples]. default 0
        [sampling]  - float - sampling frequency [Hz]. default 1.0E+09
        [file_type] - 0 or 1 - waveform or pattern respectively.
                               default pattern
    """
    
    def __init__(self, name, pulse_seq, offset=0, file_type='PAT',
                 sampling=1e9,):
        self.state = 'compiling'
        Thread.__init__(self)
        
        if file_type in [0, 'WFM']:
            self.file_type = 0
            self.name = name + '.WFM'
        elif file_type in [1, 'PAT']:
            self.file_type = 1
            self.name = name + '.PAT'
        else:
            self.file_type = 1
            self.name = name + '.PAT'
        self.index = 0
        self.pulse_seq = deepcopy(pulse_seq)
        if isinstance(self.pulse_seq, Pulse):
            self.pulse_seq = [self.pulse_seq]
        self.offset = offset
        self.sampling = sampling
        
        d = sum([int(p.duration) for p in self.pulse_seq])
        if d < 256:
            self.stub = 256 - d
        else:
            self.stub = (4 - d) % 4
        self.duration = d + self.stub
        
        self.start()

    def run(self):
        # generate header
        if self.file_type == 0:
            num_bytes = str(int(5 * self.duration))
            num_digits = str(len(num_bytes))
            header = 'MAGIC 1000\r\n#' + num_digits + num_bytes
        elif self.file_type == 1:
            num_bytes = str(int(2 * self.duration))
            num_digits = str(len(num_bytes))
            header = 'MAGIC 2000\r\n#' + num_digits + num_bytes
        # generate trailer
        trailer = 'CLOCK %16.10e' % self.sampling
        
        # generate body (data)
        wave = np.zeros(self.duration, dtype=np.float64) # use 8 byte precision for compilation
        marker1_seq = np.zeros(self.duration, dtype=np.int8)
        marker2_seq = np.zeros(self.duration, dtype=np.int8)
        t_0 = self.offset
        i = 0
        for pulse in self.pulse_seq:
            pulse.duration = int(pulse.duration)
            wave[i:i+pulse.duration] = pulse.compile(t_0)
            marker1_seq[i:i+pulse.duration] = [pulse.marker1,] * pulse.duration
            marker2_seq[i:i+pulse.duration] = [pulse.marker2,] * pulse.duration
            t_0 += pulse.duration
            i += pulse.duration
        
        # store data
        self.wave = wave
        self.marker1 = marker1_seq
        self.marker2 = marker2_seq
        
        # convert all to AWG-ready buffer
        # if waveform file:
        if self.file_type == 0:
            # create 5 byte data type
            # => 4 byte floating point data for waveform samples and 1 byte for markers
            dt = np.dtype([('w', '<f4'), ('m', '|i1')])
            
            # convert wave to 32 bit floating point data
            wave = np.float32(wave)
            # encode markers on first two LSB of a byte
            marker = marker1_seq + (marker2_seq << 1)
            
            # fill array with data
            data = np.array(zip(wave, marker), dtype=dt)
        
        # if pattern file:
        elif self.file_type == 1:
            # convert wave to 10 bit DAC channel data (encoded on 10 LSB of 16 bit int)
            wave = np.int16(np.round((self.wave + 1) * 511.5))
            data = np.zeros(self.duration, dtype=np.int16)
            # encode marker1 on bit 13 and marker2 on bit 14
            data = wave | ((marker1_seq.astype(np.int16) & 1) << 13) | ((marker2_seq.astype(np.int16) & 1) << 14)
        
        # convert to read-only buffer with header and trailer
        self.data = buffer(header + data.data[:] + trailer)
        
        # finish thread
        self.state = 'ready'
        
    # ____________
    # file-like; ftplib interface
    
    def read(self, bytes=None):
        if bytes is not None:
            r = self.data[self.index:self.index + bytes]
            self.index += bytes
        else:
            r = self.data[self.index:]
            self.index = len(self.data)
        return r
        
    def seek(self, byte):
        self.index = byte
        
    def close(self):
        pass
        
    # ____________
    # user interface
    
    def __repr__(self):
        string = '<awg520.Waveform> ' + self.name
        return string
        
    def __str__(self):
        self.seek(0)
        return self.read()
        
    def save(self, filename=None, mode='b'):
        """Save waveform in CWD."""
        if filename is None:
            filename = self.name
        if mode == 'b':
            self.seek(0)
            with open(filename, 'w') as wfm:
                wfm.write(self.read())
        elif mode == 'a':
            filename = self.name[:-4] + '.DAT'
            with open(filename, 'w') as dat:
                for i in range(self.duration):
                    sample  = str(self.wave[i])
                    marker1 = str(self.marker1[i])
                    marker2 = str(self.marker2[i])
                    # write
                    line = ' '.join([sample, marker1, marker2]) + '\n'
                    dat.write(line)
        
    def plot(self):
        """ Create a simple plot.
        
            Do not use this in a running enthought application. The wx-app of
            matplotlib will cause enthought to crash.
        """
        plt.figure()
        i = 0
        for p in self.pulse_seq:
            i += p.duration
            plt.plot([i,i], [-1,1], 'r--')
        plt.step(np.arange(len(self.wave)) + 1, self.wave, 'k')
        plt.xlim((0,len(self.wave)))
        plt.ylim((-1,1))
        plt.plot((0,len(self.wave)),(0,0), 'k--')
        plt.show()

    def plotAll(self):
        plt.figure()
        plt.subplot(311)
        i = 0
        for p in self.pulse_seq:
            i += p.duration
            plt.plot([i,i], [-1,1], 'r--')
        plt.step(np.arange(len(self.wave)) + 1, self.wave, 'k')
        plt.xlim((0,len(self.wave)))
        plt.ylim((-1,1))
        plt.plot((0,len(self.wave)),(0,0), 'k--')
        plt.show()

        mkr1 = np.zeros(len(self.wave))
        mkr2 = np.zeros(len(self.wave))

        i = 0
        for p in self.pulse_seq:
            if p.marker1:
                mkr1[i:i+p.duration] = np.ones(p.duration)
            if p.marker2:
                mkr2[i:i+p.duration] = np.ones(p.duration)
            i += p.duration

        plt.subplot(312)
        plt.xlim(0,len(self.wave))
        plt.ylim(-0.1, 1.1)
        plt.plot(mkr1)
        plt.subplot(313)
        plt.xlim(0,len(self.wave))
        plt.ylim(-0.1, 1.1)
        plt.plot(mkr2)

        
    def splitWave(self,entries=4194048):
        entries = 50
        waves=[]
        for i in range(0,len(self.wave),entries):
            w=ArbWaveform(self.name+str(i),self.wave[i*entries:i+1*entries],self.marker1[i*entries:i+1*entries],self.marker2[i*entries:i+1*entries],sampling=self.sampling)
            w.join()
            waves.append(w)
        return waves
        
class ArbWaveform(Waveform):
    def __init__(self,name,pulse,marker1,marker2,sampling=1e9):
        self.state = 'compiling'
        Thread.__init__(self)
        self.name=name
        self.wave=deepcopy(pulse)
        self.marker1=marker1
        self.marker2=marker2
        self.duration=len(self.wave)
        if self.duration < 256:
            self.stub = 256 - self.duration
        else:
            self.stub = (4 - self.duration) % 4
        self.duration = self.duration + self.stub
        
    def run(self):
        # generate header
        if self.file_type == 0:
            num_bytes = str(int(5 * self.duration))
            num_digits = str(len(num_bytes))
            header = 'MAGIC 1000\r\n#' + num_digits + num_bytes
        elif self.file_type == 1:
            num_bytes = str(int(2 * self.duration))
            num_digits = str(len(num_bytes))
            header = 'MAGIC 2000\r\n#' + num_digits + num_bytes
        # generate trailer
        trailer = 'CLOCK %16.10e' % self.sampling
        
        # generate body (data)
        wave =self.wave
        marker1_seq = self.marker1
        marker2_seq = self.marker2

        
        # convert all to AWG-ready buffer
        # if waveform file:
        if self.file_type == 0:
            # create 5 byte data type
            # => 4 byte floating point data for waveform samples and 1 byte for markers
            dt = np.dtype([('w', '<f4'), ('m', '|i1')])
            
            # convert wave to 32 bit floating point data
            wave = np.float32(wave)
            # encode markers on first two LSB of a byte
            marker = marker1_seq + (marker2_seq << 1)
            
            # fill array with data
            data = np.array(zip(wave, marker), dtype=dt)
        
        # if pattern file:
        elif self.file_type == 1:
            # convert wave to 10 bit DAC channel data (encoded on 10 LSB of 16 bit int)
            wave = np.int16(np.round((self.wave + 1) * 511.5))
            data = np.zeros(self.duration, dtype=np.int16)
            # encode marker1 on bit 13 and marker2 on bit 14
            data = wave | ((marker1_seq.astype(np.int16) & 1) << 13) | ((marker2_seq.astype(np.int16) & 1) << 14)
        
        # convert to read-only buffer with header and trailer
        self.data = buffer(header + data.data[:] + trailer)
        
        # finish thread
        self.state = 'ready'

In [19]:
class AWG520(object):
    """Controller for the Tektronix AWG520 device.
    
    SCPI commands are issued via gpib.
    See device manual for command documentation.
    File management is done via FTP.
    
    """
    
    def __init__(self, gpib='GPIB0::4',  # ::INSTR',
                       ftp='129.69.46.81',
                       socket=('129.69.46.81', 4000)):
        self.socket_addr = socket
        # set ftp parameters
        self.ftp_addr = ftp
        self.ftp_user = '\r'
        self.ftp_pw = '\r'
        self.ftp_cwd = '/main/waves'
        # self.ftp_manager = FTPManager(self)
        self.todo = -1
        self.done = -1
        # setup gpib connection
        self.gpib_addr = gpib
        self.rm = visa.ResourceManager()
        self.gpib = self.rm.open_resource(self.gpib_addr)
        self.gpib.timeout = 120000
        self.gpib_lock = threading.Lock()
        self.upload_timeout = 3.
        self.abort = False
        
    def connect_ftp(self):
        connection_trial_number = 0
        while connection_trial_number < 2:
            try:
                self.ftp = FTP(self.ftp_addr, timeout=3.0)
                connection_trial_number = 5
            except:
                print("ftp error:", sys.exc_info()[0])
                print("FTP connection did not work, retry")
                connection_trial_number += 1
        self.ftp.set_pasv(False)
        self.ftp.login('a', 'a')
        self.ftp.cwd(self.ftp_cwd)
        
    def disconnect_ftp(self):
        self.ftp.quit()

    def __del__(self):
        self.gpib.close()

    def reconnect_gpib(self):
        self.gpib.close()
        self.gpib =self.rm.open_resource(self.gpib_addr)
        self.gpib.timeout = 240000
    
    # ____________
    # File Management

    def upload(self, files):
        if not isinstance(files, (list, tuple)):
            files = [files]
        if isinstance(files[0], waveform.Waveform):
            # print "upload using ftp"
            # self.upload_ftp(files)
            print("Uploading with the GPIB . . .")
            self.upload_gpib(files)  # ftp should be better, but crashed just recently
        elif isinstance(files[0], waveform.Sequence):
            # print "upload using gpib"
            self.upload_gpib(files)
        else:
            print("unknown filetype to upload")
            raise IOError

    def upload_ftp(self, files):
        # allow single files
        uploaded = False
        count = 0
        while not uploaded and count < 10:
            try:
                self.connect_ftp()
                uploaded = True
            except:
                print("ftp error:", sys.exc_info()[0])
                uploaded = False
                count += 1
                break
            if not isinstance(files, (list, tuple)):
                files = [files]

            # opens up new ftp connections in separate threads
            self.todo = len(files)
            self.done = 0
            for i, fil in enumerate(files):
                if self.abort:
                    break
                print('\rUploading '+fil.name + ' to AWG ('+str(i+1)+' of '+str(len(files))+')'),
                try:
                    self.ftp.storbinary('STOR ' + fil.name, fil)
                except:
                    print("ftp error:", sys.exc_info()[0])
                    return
            print("")
            self.disconnect_ftp()
        
    def delete(self, files):
        self.connect_ftp()
        if not isinstance(files, (list, tuple)):
            files = [files]
    
        # opens up new ftp connections in separate threads
        self.todo = len(files)
        self.done = 0
        for i, fil in enumerate(files):
            print('Deleting ' + fil + ' from AWG (' + str(i) + ' of ' + str(len(files)) + ')')
            self.ftp.voidcmd('DELE ' + fil)
        self.disconnect_ftp()

    def upload_gpib(self, files):
        if not isinstance(files, (list, tuple)):
            files = [files]
    
        self.todo = len(files)
        self.done = 0
        for i, fil in enumerate(files):
            if self.abort:
                break
            print('\rUploading '+fil.name + ' to AWG ('+str(i+1)+' of '+str(len(files))+')'),
            _timer = 0.
            # print "gpib upload process"
            while not fil.state == 'ready':
                time.sleep(0.1)
                _timer += 0.1
                print(r"\r not ready, _timer is " + str(_timer)),
                if _timer >= self.upload_timeout:
                    print('Timed out.')
                    break
            self.tell_raw('MMEM:DATA "\\waves/'+fil.name+'",#'+str(len(str(len(str(fil)))))+''+str(len(str(fil)))+''+str(fil))
            time.sleep(.1)
        print("")
            
    def delete_gpib(self, files):
        if not isinstance(files, (list, tuple)):
            files = [files]
        for fil in files:
            if type(fil) == str:
                self.tell('MMEM:DEL "\\waves/'+fil+'","MAIN"')
            else:
                self.tell('MMEM:DEL "\\waves/'+fil.name+'","MAIN"')
            time.sleep(.1)
    
    def delete_all(self):
        """Remove all files from the AWG's CWD.
        """
        self.connect_ftp()
        fl = self.ftp.nlst()
        try:
            fl.remove('.')
            fl.remove('..')
        except:
            pass
        print(fl)
        self.delete(fl)
        self.disconnect_ftp()

    # ____________
    # Operation Commands
    
    def tell(self, command):
        """Send a command string to the AWG."""
        self.gpib.write(command)
        time.sleep(0.5)

    def tell_raw(self, command):
        """Send a command string to the AWG."""
        self.gpib.write_raw(command)
        time.sleep(0.5)
        
    def ask(self, query):
        """Send a query string to AWG and return the response."""
        self.gpib.write(query)
        try:
            res = self.gpib.read()
        except visa.VisaIOError as e:
            res = ''
            if 'Timeout' in e.message:
                res = 'No response from AWG for: "' + query + '"'
            else:
                raise e
        finally:
            time.sleep(0.5)
        return res
    
    def run(self):
        self.tell('AWGC:RUN')
    
    def stop(self):
        self.tell('AWGC:STOP')
        
    def get_state(self):
        # 0: Stopped, 1:waiting for trigger, 2: running
        return self.ask('AWGC:RST?')
        
    def get_mode(self):
        return self.ask('AWGC:RMOD?')
        
    def get_output(self):
        ch1 = self.ask('OUTP1?')
        ch2 = self.ask('OUTP2?')
        if ch1 == '0' and ch2 == '0':
            return 0b00
        elif ch1 == '1' and ch2 == '0':
            return 0b01
        elif ch1 == '0' and ch2 == '1':
            return 0b10
        else:
            return 0b11
        
    def force_trigger(self):
        self.tell('*TRG')
        self.ask('*OPC?')

    def set_output(self, channel=0b11):
        """Set the output state of specified channels.
        
        channels - int with states encoded on 2 LSB
                   e.g. bit=0b00 closes all, bit=0b11 opens all,
                        bit=0b10 opens OUTP2 and closes OUTP1
        
        """
        for i in [0, 1]:
            stat = channel >> i & 1 
            self.tell('OUTP%i %i' % ((i+1), stat))
        
    def set_mode(self, mode):
        """Change the output mode.
        
        Options for mode (case-insensitive):
        continuous - 'C', 'CONT'
        triggered  - 'T', 'TRIG'
        gated      - 'G', 'GAT'
        sequence   - 'S', 'SEQ'
        
        """
        look_up = {'C': 'CONT', 'CON': 'CONT', 'CONT': 'CONT',
                   'T': 'TRIG', 'TRI': 'TRIG', 'TRIG': 'TRIG',
                   'G': 'GAT', 'GAT': 'GAT', 'GATE': 'GAT',
                   'E': 'ENH', 'ENH': 'ENH', 'ENHA': 'ENH',
                  }
        self.tell('AWGC:RMOD %s' % look_up[mode.upper()])
    
    def set_sampling(self, frequency):
        """ Set the output sampling rate.
        
        """
        frequency *= 1e-9
        self.tell('SOUR:FREQ %.4GGHz' % frequency)
    
    def set_vpp(self, voltage, channel=0b11):
        """ Set output peak-to-peak voltage of specified channel.
            
        """
        if channel & 1 == 1:
            self.tell('SOUR1:VOLT %.4GV' % voltage)
        if channel & 2 == 2:
            self.tell('SOUR2:VOLT %.4GV' % voltage)
            
    def get_vpp(self, channel):
        if channel & 1 == 1:
            return float(self.ask('SOUR1:VOLT?'))
        else:
            return float(self.ask('SOUR2:VOLT?'))
    
    def load(self, filename, channel=1, cwd='\waves', block=False):
        """Load sequence or waveform file into RAM, preparing it for output.
        
        Waveforms and single channel sequences can be assigned to each or both
        channels. Double channel sequences must be assigned to channel 1.
        The AWG's file system is case-sensitive.
        
        """
        if hasattr(filename, 'name'):
            filename = filename.name
        self.tell('SOUR%i:FUNC:USER "%s/%s"' % (channel, cwd, filename))
        
        # block thread until the operation is complete
        while block:
            try:
                self.ask('*OPC?')
                # self.tell('SYST:BEEP')
                block = False
            except visa.VisaIOError as e:
                if not 'Timeout' in e[0]:
                    raise e
    
    def managed_load(self, filename, channel=1, cwd='\waves'):
        self.ftp_manager.load(filename, channel, cwd)
        
    def get_func(self, channel=1):
        res = self.ask('SOUR%i:FUNC:USER?' % channel)
        # res ~ '"/\\waves/0_MAIN.SEQ","MAIN"'
        return res.split(',')[0].split('/')[-1][:-1]  # return ~ '0_MAIN.SEQ'
        
    def reset(self):
        """ Reset the AWG settings. """
        self.tell('*RST')
        
    def resetSequencePosition(self, line=1):
        self.tell('AWGC:EVEN:SOFT:IMM '+str(line))
        self.ask('*OPC?')
        

# class FTPThread(threading.Thread):
#    """ Thread, which opens a new FTP connection.
#    """
#    def __init__(self, awg):
#        # emulate state stuff
#        class Foo(): pass
#        self.file = Foo()
#        self.file.state = 'compiling'
#
#        self.awg = awg
#        super(FTPThread, self).__init__()
#        self.daemon = True
#        self.file.state = 'ready'
#
#    def setup_ftp(self):
#        self.ftp = FTP(self.awg.ftp_addr, timeout=2.0)
#        self.ftp.set_pasv(False)
#        self.ftp.login(self.awg.ftp_user, self.awg.ftp_pw)
#        self.ftp.cwd(self.awg.ftp_cwd)
#
#    def run(self):
#        try:
#            self.setup_ftp()
#            self.task()
#            self.ftp.close()
#            self.file.state = 'finished'
#        except Exception as e:
#            try:
#                self.ftp.close()
#            except AttributeError as a:
#                pass
#            self.file.state = 'error'
#            # dont raise error_temp
#            # if not isinstance(e, error_temp):
#            raise e
#
#    def task(self): pass
#
# class UploadThread(FTPThread):
#
#    def __init__(self, awg, file):
#        super(UploadThread, self).__init__(awg)
#        self.file = file
#
#    def task(self):
#        self.file.seek(0)
#        self.ftp.storbinary('STOR ' + self.file.name, self.file)
#
# class DeleteAllThread(FTPThread):
#
#    def task(self):
#        filelist = self.ftp.nlst()
#        try:
#            filelist.remove('.')
#            filelist.remove('..')
#        except ValueError:
#            pass
#        self.awg.tell('MMEM:CDIR "%s"' % self.awg.ftp_cwd)
#        for file in filelist:
#            self.awg.tell('MMEM:DEL "%s"' % file)
#            #self.ftp.delete(file)
#        time.sleep(0.5)
#
# class FTPManager(threading.Thread):
#    """ This Thread will prevent/workaround 421 session limit.
#
#        It is only able to do to tasks, uploading files and deleting all files.
#    """
#
#    def __init__(self, awg):
#        self.awg = awg
#        self.threads = []
#        self.clients = 0
#        self.max_clients = 1
#        self.awg.done = -1
#        self.awg.todo = -1
#        self.abort = False
#        self.pause_set = False
#        self.paused = False
#        self.load_file = None
#        super(FTPManager, self).__init__()
#        self.daemon = True
#        self.start()
#
#    def upload(self, file):
#        ut = UploadThread(self.awg, file)
#        self.threads.append(ut)
#
#    def delete_all(self):
#        dt = DeleteAllThread(self.awg)
#        self.threads.append(dt)
#
#    def load(self, filename, channel=1, cwd='\waves'):
#        self.load_file = { 'filename': filename,
#                           'channel' : channel,
#                           'cwd'     : cwd
#                         }
#
#    def reset(self):
#        self.pause_set = True
#        self.threads = []     # really bad practice!!! TODO: make stappable threads - stop and join them
#
#        while not self.paused:
#            time.sleep(0.1)
#        self.clients = 0
#        self.awg.done = -1
#        self.awg.todo = -1
#        self.pause_set = False
#
#    def stop(self):
#        self.abort = True
#
#    def run(self):
#        # Event loop
#        while True:
#            # check list of threads repeatedly
#            for thr in self.threads:
#                if self.abort: return
#                # ignore running threads
#                if not thr.is_alive():
#
#                    # Case DeleteAllThread:
#                    if isinstance(thr, DeleteAllThread):
#                        # start a DeleteAllThread
#                        if thr.file.state == 'ready' and self.clients == 0:
#                            thr.start()
#                            self.clients += self.max_clients
#                            #time.sleep(0.001)
#                        # remove finished DeleteAllThread
#                        elif thr.file.state == 'finished':
#                            self.clients = 0
#                            self.threads.remove(thr)
#                        # restart failed DeleteAllThread
#                        elif thr.file.state == 'error':
#                            self.clients = 0
#                            self.threads.remove(thr)
#                            self.delete_all()
#
#                    # Case UploadThread:
#                    elif isinstance(thr, UploadThread):
#                        # start new UploadThread
#                        if thr.file.state == 'ready' and self.clients < self.max_clients:
#                            thr.start()
#                            self.clients += 1
#                            #time.sleep(0.001)
#                        # remove finished UploadThread
#                        elif thr.file.state == 'finished':
#                            self.clients -= 1
#                            self.threads.remove(thr)
#                            self.awg.done += 1
#                        # restart failed UploadThread
#                        elif thr.file.state == 'error':
#                            self.clients -= 1
#                            thr.file.seek(0)
#                            thr.file.state = 'ready'
#                            self.upload(thr.file)
#                            self.threads.remove(thr)
#                # stop threads if abort is set
#                time.sleep(0.001)
#            # check if there is something to load into RAM
#            if len(self.threads) == 0 and self.awg.done != -1 and self.load_file is not None:
#                f = self.load_file
#                self.awg.load(f['filename'], f['channel'], f['cwd'], block=True)
#                self.load_file = None
#            if self.pause_set:
#                self.paused = True
#                while self.pause_set:
#                    time.sleep(0.1)
#                self.paused = False
#
#

class AWGHasTraits( HasTraits, AWG520 ):
    
    todo         = Int()
    done         = Int()
    progress     = Property(trait=Float, depends_on=['todo', 'done'],  format_str='%.1f')
    abort_button = Button(label='abort upload')
    
    en1  = Bool(False, label='CH 1', desc='enable CH1')
    en2  = Bool(False, label='CH 2', desc='enable CH2')
    
    current_wave = Str('', label='Wave', desc='last set waveform or sequence')
    
    mode = Enum(['ENH', 'CONT', 'TRIG', 'GATE'], desc='select run mode', editor=EnumEditor(values=['CONT', 'TRIG', 'GATE', 'ENH'], cols=4, format_str='%s'))
    
    run_button  = Button(label='Run', desc='Run')
    stop_button = Button(label='Stop', desc='Stop')
    trig_button = Button(label='Trigger', desc='Trigger')
    
    def __init__(self, gpib='GPIB0::2::INSTR',
                       ftp='192.168.0.3',
                       socket=('192.168.0.3',4000),
                       **kwargs
                ):
        AWG520.__init__(self, gpib, ftp, socket)
        HasTraits.__init__(self, **kwargs)
    
    def sync_upload_trait(self, client, trait_name):
        self.sync_trait('progress', client, trait_name, mutual=False)
    
    def _get_progress(self):
        return (100.0 * self.done) / self.todo
        
    def _abort_button_fired(self):
        self.ftp_manager.reset()
        
    def _run_button_fired(self):
        self.run()
        
    def _stop_button_fired(self):
        self.stop()
        
    def _trig_button_fired(self):
        self.force_trigger()
        
    @on_trait_change('en1', 'en2')
    def change_output(self):
        self.set_output((self.en2 << 1) + self.en1)
        
    @on_trait_change('mode')
    def change_mode(self):
        self.set_mode(self.mode)
    
    #override
    def load(self, filename, channel=1, cwd='\waves', block=False):
        self.current_wave = filename
        super(AWGHasTraits, self).load(filename, channel, cwd, block)
    
    view = View(VGroup(HGroup(Item('progress', width=40, style='readonly', format_str='%.1f'),
                              Item('todo', width=40, style='readonly'),
                              Item('abort_button', show_label=False),
                              label='FTP',
                              show_border=True,
                             ),
                       VGroup(HGroup(Item('en1'),
                                     Item('en2'),
                                     Item('run_button', show_label=False),
                                     Item('stop_button', show_label=False),
                                     Item('trig_button', show_label=False),
                                    ),
                              HGroup(Item('mode', style='custom', show_label=False, width=-250),
                                     Item('current_wave', style='readonly')
                                    ),
                              label='Output',
                              show_border=True,
                             ),
                      ),
                title='AWG520', width=550, buttons=[], resizable=True
               )
# _____________________________________________________________________________
# EXCEPTIONS:
    # TODO

# _____________________________________________________________________________
# DEBUG SCRIPT:

if __name__ == '__main__':
    pass

NameError: name 'HasTraits' is not defined

In [18]:
print("ftp error:")

ftp error:
